In [ ]:
import random
from collections import defaultdict
from scipy import stats, linalg
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import pandas as pd
from pandas import DataFrame, read_csv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import hist, scatter

In [ ]:
DATA_CSV = 'data/forest.csv'
data_points = pd.read_csv(DATA_CSV, index_col='index')
orig_cols = data_points.columns

In [ ]:
#Apply Z-score normalization
for column in data_points:
    data_points[column] = stats.zscore(data_points[column], axis=None)

In [ ]:
#Apply [0,1] normalization
for column in data_points:
    data_points[column] = (
        data_points[column] - min(data_points[column])
    ) / (
        max(data_points[column]) - min(data_points[column])
    )

In [ ]:
#Apply PCA, create the clusters and add columns to ease graphing
pca = PCA(n_components='mle')
data_pca = pca.fit_transform(data_points)
N_CLUSTERS = 20
clusters = KMeans(n_clusters=N_CLUSTERS).fit(data_pca)
centroids = clusters.cluster_centers_
labels = clusters.labels_
data_points['clusters'] = labels
data_points['plot'] = data_points.index.values / 10000
data_points['year'] = data_points.index.values % 10000
data_points['color'] = [sns.color_palette('cubehelix',N_CLUSTERS)[x] for x in data_points['clusters']]

In [ ]:
#Reset dataframe if you want to re-cluster
data_points = data_points.loc[:, orig_cols]

In [ ]:
g = sns.lmplot(x='iv97.0', y='iv375.0', data=data_points, hue='iv241.0', col='clusters_c', row='clusters_r', palette=sns.color_palette("Reds_d") , legend=False, fit_reg=False)
g.show()

In [ ]:
g = sns.pointplot(x='year', y='iv375.0', data=data_points.iloc[0:6], hue='plot',  palette=sns.color_palette('cubehelix',5), join=True)
g.show()

In [ ]:
g = sns.factorplot(x='iv97.0', y='iv375.0', hue='plot', data=data_points.iloc[0:19])
g.show()

In [ ]:
g = sns.PairGrid(data_points, hue = 'clusters')
g = g.map_diag(plt.hist)
g = g.map_offdiag(plt.scatter)
g.show()

In [ ]:
SUB_FRAME = data_points.iloc[765:798]
X = 'ba'
Y = 'iv375.0'
for key,grp in SUB_FRAME.groupby('plot'): 
    plt.plot(grp[X], grp[Y],'k-', label = key) 
    for i, row in grp.iterrows(): 
        plt.plot(row[X], row[Y], 'o', markerfacecolor = row['color'])
plt.xlabel(X)
plt.ylabel(Y)
sns.plt.show()